General outline:
I want to take in an array (later image) representation of a scrabble board/hand tiles, and output the best play.

Naive best play: the highest scoring play

Goal best play: the play that maximizes your winning chances (takes into account the opponent's potential score)
    * You want to maximize how you're using the tiles in your hand. 
        * This doesn't just mean getting the highest score possible. Save your high tiles for the multipliers.
    * As well as minimizing your opponent's reply score

0. Constants:
    * Multiplier boards
    
* make functions that define the area that the word is in on the board (given a letter?)
    * allow another function to tell what the letters are in that area.


Main steps needed:
1. Take in an array of the board and your hand letters **check**
2. Tell the available permutations and their locations on the board **check**
    * return a dictionary with the word as a key and the location as the value **check**
3. See what crossing words those permutations are going to make **check**
    * this will return a dictionary of the main word and its position as a key and the crossing words as the values **check**
    * could use a nested dictionary for this **check**
4. Check to see if the permutations and their crossing words are in the dictionary **check**
    * take in the dictinary from above? **check**
5. Do the same for the edge words.
6. Score it.
For 

Step 2:
1. Need to get permutations for each row
       * Options:
           1. Get permutations around each group of letters like I've been doing
           2. Look at it on a row-wide basis **Went with this one**
               * Need to make sure all the sample letters stay together

TODO: it would probably be best to check the crossing words fewer times. Perhaps I could make a set of them, find the intersection of them and the dictionary, then loop over that intersection with the individual groups.

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import csv
import itertools
import re
from collections import defaultdict


<IPython.core.display.Javascript object>

In [121]:
sample_board = [
    ["", "", "", "", "", "", "", "", "", "", "d", "e", "a", "d", ""],  # 0
    ["", "", "", "", "", "", "", "", "", "z", "o", "n", "a", "", "w"],  # 1
    ["", "", "", "", "", "", "", "", "", "", "", "a", "", "", "i"],  # 2
    ["", "", "", "", "", "", "", "", "", "r", "i", "m", "a", "", "l"],  # 3
    ["", "", "", "", "", "", "", "", "", "", "", "o", "b", "e", "y"],  # 4
    ["", "", "", "", "", "", "", "", "", "j", "u", "r", "a", "l", ""],
    ["", "", "", "", "", "", "", "b", "e", "e", "p", "", "", "", ""],  # 6
    ["", "", "", "f", "", "", "y", "e", "w", "s", "", "t", "", "", ""],
    ["", "", "", "o", "", "", "", "l", "e", "t", "c", "h", "", "", ""],  # 8
    ["", "", "", "c", "h", "u", "r", "l", "", "", "", "i", "", "r", ""],
    ["", "", "g", "i", "", "", "", "e", "", "", "", "g", "", "e", ""],  # 10
    ["", "", "i", "", "", "", "", "n", "", "", "", "h", "u", "e", "d"],
    ["", "", "n", "", "", "", "", "d", "", "", "", "", "", "f", ""],  # 12
    ["", "", "", "", "", "", "", "", "", "v", "e", "e", "p", "s", ""],
    ["", "", "", "", "", "", "", "", "n", "u", "n", "", "", "", ""],  # 14
]
sample_board_letters = ["o", "a", "s", "r", "l",'r','t']

# numpy of the board
board_array = np.array(sample_board)

# making the empty strings spaces
neat_sample_array = np.copy(board_array)
neat_sample_array[neat_sample_array == ""] = " "


<IPython.core.display.Javascript object>

In [134]:
class Scrabbler:
    data_path = "../sowpods.txt"
    with open(data_path, newline="") as f:
        reader = csv.reader(f)
        allwords = list(reader)
    WORDLIST = list(itertools.chain.from_iterable(allwords))
    WORDLIST = [x.lower() for x in WORDLIST]
    WORDSET = set(WORDLIST)
    BOARD_LENGTH = 15
    MAX_I = 14
    row_mainword_scores = {}
    column_mainword_scores = {}

    LETTER_MULTIPLIERS = [
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 0
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],  # 1
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 2
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 3
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 4
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 6
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 8
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 10
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 12
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 14
    ]
    LETTER_MULTIPLIERS_ARRAY = np.array(LETTER_MULTIPLIERS)

    WORD_MULTIPLIERS = [
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 0
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],  # 1
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 2
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],  # 3
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 4
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 6
        [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 8
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 10
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 12
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 14
    ]
    WORD_MULTIPLIERS_ARRAY = np.array(WORD_MULTIPLIERS)

    TILE_SCORES = {
        "a": 1,
        "b": 4,
        "c": 4,
        "d": 2,
        "e": 1,
        "f": 4,
        "g": 3,
        "h": 3,
        "i": 1,
        "j": 10,
        "k": 5,
        "l": 2,
        "m": 4,
        "n": 2,
        "o": 1,
        "p": 4,
        "q": 10,
        "r": 1,
        "s": 1,
        "t": 1,
        "u": 2,
        "v": 5,
        "w": 4,
        "x": 8,
        "y": 3,
        "z": 10,
    }

    def __init__(self, board, hand_letters):
        self.board = board
        self.hand_letters = hand_letters
        self.T_board = np.copy(board).transpose()  # TODO: should this be here?
        self.board_mask = (np.copy(board) != "").astype(int)

    # Navigator functions
    def check_yx(self, y, x):
        if x > -1 and x < 15 and y > -1 and y < 15:
            return bool(self.board[y, x])
        else:
            return "out"

    def check_right(self, y, x):
        if x < self.BOARD_LENGTH - 1:
            return bool(self.board[y, x + 1]), self.board[y, x + 1]
        else:
            return "out"

    def check_left(self, y, x):
        if x > 0:
            return bool(self.board[y, x - 1]), self.board[y, x - 1]
        else:
            return "out"

    def check_above(self, y, x):
        if y > 0:
            return bool(self.board[y - 1, x]), self.board[y - 1, x]
        else:
            return "out"

    def check_below(self, y, x):
        if y < self.BOARD_LENGTH - 1:
            return bool(self.board[y + 1, x]), self.board[y + 1, x]
        else:
            return "out"

    # For potential words of a given length
    # start at the left, travel right (keep track of empties) till you hit a letter
    def hand_letter_permutations(self, length):
        # this returns a list of permutations of the hand letters of a given length
        letter_permutations = list(itertools.permutations(self.hand_letters, length))
        return letter_permutations

    def row_permutations(self, row):
        y = row
        board_row = self.board[row]

        # no_ means "number of" here
        no_hand_letters = len(self.hand_letters)
        no_pre_filled = sum(list(map(bool, board_row)))
        row_perms_dict = {}

        if no_pre_filled > 0:
            for perm_len in range(1, no_hand_letters + 1):
                hand_perms = self.hand_letter_permutations(perm_len)
                # for x in range(0, BOARD_LENGTH-(len(self.hand_letters)-1)-no_pre_filled):
                # x=0
                no_tiles_to_right = no_pre_filled
                for hand_perm in hand_perms:
                    # while x + no_tiles_to_right+perm_len<self.BOARD_LENGTH:
                    x = 0
                    if (
                        self.check_yx(y, x) == False
                    ):  # and sum(list(map(bool,board_row[x:x+no_hand_letters])))
                        # for hand_perm in hand_perms:
                        while True:
                            # TODO check this
                            if (
                                any(
                                    board_row[max(0, x - 1) : min(x + perm_len + 1, 14)]
                                )
                                == True
                            ):
                                complete_word, word_y, word_x = self.complete_word_y_x(
                                    row, x, hand_perm
                                )
                                if complete_word == []:
                                    break
                                elif len(complete_word) > perm_len:
                                    # TODO: could leave out the part after the and below and delete duplicates later
                                    # TODO: could indicate the length of the original perm
                                    if (
                                        complete_word in row_perms_dict
                                        and [word_y, word_x]
                                        not in row_perms_dict[complete_word]
                                    ):
                                        row_perms_dict[complete_word].append(
                                            [word_y, word_x]
                                        )
                                    else:
                                        row_perms_dict[complete_word] = [
                                            [word_y, word_x]
                                        ]
                            x = x + 1
                            if x == self.BOARD_LENGTH:
                                break
                    # no_tiles_to_right=sum(list(map(bool,board_row[x+1:])))#TODO: +1 here?
        return row_perms_dict

    # fill the next len(hand_perm) spaces with hand tiles and find the connecting words
    # row=y
    def complete_word_y_x(self, row, x, hand_perm):
        hand_perm = list(hand_perm)
        perm_len = len(hand_perm)
        X = x
        board_row = self.board[row]
        y = row
        no_filled = 0
        filled_tiles = []
        left_letters = []
        right_letters = []
        while self.check_left(y, x)[0] == True:
            x = x - 1
            left_letters = left_letters + [board_row[x]]
        left_letters = left_letters[::-1]
        x = X
        while no_filled < perm_len:
            if self.check_yx(y, x) == True:
                filled_tiles = filled_tiles + [board_row[x]]
            elif self.check_yx(y, x) == "out":
                return [], [], []
            elif self.check_yx(y, x) == False:
                new_tile = hand_perm.pop(0)
                filled_tiles = filled_tiles + [new_tile]
                no_filled = no_filled + 1
            x = x + 1
        x = x - 1
        while self.check_right(y, x)[0] == True:
            x = x + 1
            right_letters = right_letters + [board_row[x]]
        complete_word_as_list = left_letters + filled_tiles + right_letters
        word_x = X - len(left_letters)
        complete_word = "".join(complete_word_as_list)
        return complete_word, y, word_x

    # Find the crossing words for a given complete_word_y_x info
    def find_crosses(self, complete_word, y, x):
        crossing_words = set()
        count = 0
        for tile in complete_word:
            # TODO: don't think I need this if part, only the last part and len(crossing_word)>1
            if (
                self.check_above(y, x)[0] == True or self.check_below(y, x)[0] == True
            ) and self.board[y, x] != tile:
                crossing_word, T_y, T_x = self.crossing_words_at_tile(tile, y, x)
                if len(crossing_word) > 1:
                    crossing_words.add((crossing_word, T_y, T_x))

            # TODO: not sure if I need all this stuff about edge_letter
            #             if self.board[y, x] != tile:
            #                 count = count + 1
            #                 edge_letter = tile
            #                 edge_x = x
            #                 edge_y = y

            x = x + 1
        # TODO should I use/fix this?
        #         if count == 1 and len(crossing_words) == 0:
        #             crossing_words.add((edge_letter, edge_y, edge_x))
        if len(crossing_words) == 0:
            crossing_words.add(("", -1, -1))
        return crossing_words

    # Enter a tile and its location and return the crosses that it makes
    def crossing_words_at_tile(self, tile, y, x):
        # We will be using the transpose board so y and x are switched
        self.board = self.board.transpose()
        y, x = x, y
        crossing_word, T_y, T_x = self.complete_word_y_x(y, x, (tile))
        self.board = self.board.transpose()
        return crossing_word, T_y, T_x

    # insert the "hook letter" into a shadow board and find the output connected to that
    # assume the board has already been transposed and you are operating with the right y and x
    def edge_perms(self, hook_letter, y, x):
        self.board[y, x] = hook_letter
        self.hand_letters.remove(hook_letter)
        shadow_row_perms = self.row_permutations(y)
        edge_perms_dict = {
            k: v
            for (k, v) in shadow_row_perms.items()
            if v[0][1] <= x
            and v[0][1] + len(k)
            >= x  # TODO: see if I really need to put out a nested list here
        }

        self.hand_letters.append(hook_letter)
        self.board[y, x] = ""
        return edge_perms_dict

    def find_valid_perms(self):
        crossing_set = set()
        board_perms_dict = {}

        for row in range(0, self.BOARD_LENGTH):
            row_perms = self.row_permutations(row)
            for complete_word in row_perms:
                if complete_word in board_perms_dict:
                    board_perms_dict[complete_word] = (
                        board_perms_dict[complete_word] + row_perms[complete_word]
                    )
                else:
                    board_perms_dict[complete_word] = row_perms[complete_word]
        perms_set = set(board_perms_dict.keys())
        valid_perms = self.WORDSET.intersection(perms_set)
        valid_perms_dict = {
            k: v for (k, v) in board_perms_dict.items() if k in valid_perms
        }
        return valid_perms_dict

    # use the perms to find crossings and check if they are all valid
    # TODO: ask to see if I can somehow check all of these sets at once so I only have to iter through once
    def check_crosses(self, valid_perm, y, x):
        crosses_with_position = self.find_crosses(valid_perm, y, x)
        crossing_words = set([i for i in zip(*crosses_with_position)][0])
        crosses_valid = crossing_words.issubset(
            self.WORDSET
        ) or crosses_with_position == {("", -1, -1)}
        return crosses_valid, crosses_with_position

    # make a dictionary of valid plays for one direction as a dictionary of words with a nested crossing dict
    # in the first entry of a list that are the values of the dictionary
    def valid_plays(self, valid_perms):
        # valid_perms = self.find_valid_perms() #TODO is this right to move to a parameter?

        valid_plays = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
        for valid_perm in valid_perms:
            potential_positions = valid_perms[valid_perm]
            for potential_position in potential_positions:
                y = potential_position[0]
                x = potential_position[1]
                crosses_valid, crosses_with_position = self.check_crosses(
                    valid_perm, y, x
                )
                if crosses_valid == True:
                    # TODO: I need to be able to add new positionf of the same cross here
                    # probably need to make another default dict here
                    for cross_with_position in crosses_with_position:
                        valid_plays[valid_perm][tuple(potential_position)][
                            cross_with_position[0]
                        ].append((cross_with_position[1:3]))
        return valid_plays

    # remember the crosses returned above have the x and y switched, as well as nested default dicts

    # Find which tiles of a word originated from the hand
    def new_tiles(self, valid_word, y, x):
        new_tiles = []
        for letter in valid_word:
            if self.check_yx(y, x) == False:
                new_tiles.append(
                    [letter, y, x]
                )  # should this really be a dictionary list?
            x = x + 1
        return new_tiles

    # Find which tiles could be used to build a crossing word
    def find_edge_hooks(self, valid_plays):
        edge_hooks = []
        for valid_word in valid_plays:
            for position in valid_plays[valid_word]:
                y = position[0]
                x = position[1]
                new_tiles = self.new_tiles(valid_word, y, x)
                if len(new_tiles) == 1:
                    edge_hooks = edge_hooks + new_tiles
        return edge_hooks

    # create a dictionary of the valid edge words
    # could probably adapt find valid perms above for this
    def valid_edge_words(self, edge_hooks):
        edge_perms_dict = {}
        for hook in edge_hooks:
            hook_letter = hook[0]
            # x and y are transposed
            y = hook[2]
            x = hook[1]
            edge_perms = self.edge_perms(hook_letter, y, x)

            for complete_word in edge_perms:
                if complete_word not in edge_perms_dict:
                    edge_perms_dict[complete_word] = edge_perms[complete_word]
                elif edge_perms[complete_word][0] not in edge_perms_dict[complete_word]:
                    edge_perms_dict[complete_word] = (
                        edge_perms_dict[complete_word] + edge_perms[complete_word]
                    )
        # copied from above, checking if perm in the dictionary
        edge_perms_set = set(edge_perms_dict.keys())
        valid_edge_perms = self.WORDSET.intersection(edge_perms_set)
        valid_edge_perms_dict = {
            k: v for (k, v) in edge_perms_dict.items() if k in valid_edge_perms
        }
        return valid_edge_perms_dict

    # Flattens the row and column dictionaries, making them more intuitive/combinable
    def unpack_dictionary(self, dict1):
        new_dict = {}
        for word in dict1:
            location_crosses = dict1[word]
            for location in location_crosses:
                crosses_dict = location_crosses[location]
                last_word_location = []
                for crossing_word in crosses_dict:
                    crossing_word_locations = crosses_dict[crossing_word]
                    for crossing_location in crossing_word_locations:
                        if len(last_word_location) > 0:
                            new_dict[(word, location)] = new_dict[
                                last_word_location
                            ] + (crossing_word, crossing_location)
                        else:
                            new_dict[(word, location)] = (
                                crossing_word,
                                crossing_location,
                            )
                        last_word_location = (word, location)
        new_new_dict = {}
        for key in new_dict:
            crossers_list = new_dict[key]
            new_new_dict[key] = tuple(zip(crossers_list, crossers_list[1:]))[::2]
        return new_new_dict

    # Returns 2 dictionaries of all the valid words and their crosses
    def all_board_words(self):
        # find the valid row plays
        valid_row_perms = self.find_valid_perms()
        valid_row_plays = self.valid_plays(valid_row_perms)

        # find the edge hooks for the column-wise edge words
        edge_hooks_for_columns = self.find_edge_hooks(valid_row_plays)

        # transpose
        self.board = self.board.transpose()

        # find the valid column plays
        valid_column_perms = self.find_valid_perms()
        valid_column_plays = self.valid_plays(valid_column_perms)

        # find the valid column edge plays
        valid_column_edge_perms = self.valid_edge_words(edge_hooks_for_columns)
        valid_column_edge_plays = self.valid_plays(valid_column_edge_perms)

        # find the edge hooks for the row-wise edge words
        edge_hooks_for_rows = self.find_edge_hooks(valid_column_plays)

        # transpose
        self.board = self.board.transpose()

        # find the valid row edge plays
        valid_row_edge_perms = self.valid_edge_words(edge_hooks_for_rows)
        valid_row_edge_plays = self.valid_plays(valid_row_edge_perms)

        # transform them all into a more readable format
        valid_row_plays = self.unpack_dictionary(valid_row_plays)
        valid_row_edge_plays = self.unpack_dictionary(valid_row_edge_plays)
        valid_column_plays = self.unpack_dictionary(valid_column_plays)
        valid_column_edge_plays = self.unpack_dictionary(valid_column_edge_plays)

        # collapse the row and column dictionaries
        all_row_words = {**valid_row_plays, **valid_row_edge_plays}
        all_column_words = {**valid_column_plays, **valid_column_edge_plays}

        # now I have two dictionaries: row and column in the format of:
        # {(mainword, y, x): ((crossingword, y, x), (crossingword, y, x), etc)
        # (mainword, y, x): ((crossingword, y, x), etc)
        # ...
        # }
        # remember that the y and x of the crossing words refer to a transposed board vis-a-vis the main word

        return all_row_words, all_column_words

    # calculate the score of a group of tiles (not the whole play)
    def calculate_word_score(
        self, word, y, x, letter_multipliers_array, word_multipliers_array
    ):
        letter_multipliers = np.copy(letter_multipliers_array[y, x : x + len(word)])
        word_multipliers = np.copy(word_multipliers_array[y, x : x + len(word)])
        scores = [self.TILE_SCORES[l] for l in word]
        word_score = np.dot(letter_multipliers, scores)
        word_score = word_score * np.prod(word_multipliers)
        return word_score

    # calculating the total value of a main word and its crosses
    def total_score(self, main_word_and_location, crossing_words_and_locations):
        letter_multipliers_array = np.copy(self.LETTER_MULTIPLIERS_ARRAY)
        word_multipliers_array = np.copy(self.WORD_MULTIPLIERS_ARRAY)

        # make the  multipliers==1 if there are pieces on them on the board
        # TODO: how permanent is the below?
        bool_mask = (self.board != "").astype(int)
        letter_multipliers_array = np.where(bool_mask == 1, 1, letter_multipliers_array)
        word_multipliers_array = np.where(bool_mask == 1, 1, word_multipliers_array)

        main_word = main_word_and_location[0]
        y = main_word_and_location[1][0]
        x = main_word_and_location[1][1]
        main_score = self.calculate_word_score(
            main_word, y, x, letter_multipliers_array, word_multipliers_array
        )

        # TODO: ask if I should have transpose inside or outside the copy function here
        # transpose the multiplier arrays to use to score the crossing words
        T_word_multipliers = np.copy(word_multipliers_array.transpose())
        T_letter_multipliers = np.copy(letter_multipliers_array.transpose())

        # calculating the crossing words' total score
        crossers_score = 0
        for crossing_word_and_location in crossing_words_and_locations:
            if len(crossing_word_and_location[0]) > 1:
                y = crossing_word_and_location[1][0]
                x = crossing_word_and_location[1][1]
                crossing_word = crossing_word_and_location[0]
                crosser_score = self.calculate_word_score(
                    crossing_word, y, x, T_letter_multipliers, T_word_multipliers
                )
            else:
                crosser_score = 0
            crossers_score = crossers_score + crosser_score
        total_score = main_score + crossers_score
        return total_score

    # Create a dictionary of all valid main words and their scores
    def valid_score_choices(self):
        board = self.board
        hand_letters = self.hand_letters
        letter_multiplier_array = self.LETTER_MULTIPLIERS_ARRAY
        word_multiplier_array = self.WORD_MULTIPLIERS_ARRAY

        # make a transpose board
        transpose_board = np.copy(board)
        transpose_board = transpose_board.transpose()

        # calculate the words/placement allowed by the rules of the game
        all_row_words, all_column_words = self.all_board_words()

        # create a dictionary of each of these row words along with their score
        row_score_dict = {}
        column_score_dict = {}
        for main_word_and_location in all_row_words:
            crossing_words_and_locations = all_row_words[main_word_and_location]
            score = self.total_score(
                main_word_and_location, crossing_words_and_locations
            )
            row_score_dict[main_word_and_location] = score

        # transpose the board
        self.board = self.board.transpose()

        # same for the column words
        for main_word_and_location in all_column_words:
            crossing_words_and_locations = all_column_words[main_word_and_location]
            score = self.total_score(
                main_word_and_location, crossing_words_and_locations
            )
            column_score_dict[main_word_and_location] = score

        self.row_mainword_scores = row_score_dict
        self.column_mainword_scores = column_score_dict
        return row_score_dict, column_score_dict

    # Return the highest scoring k words for rows and columns, if you have already run total_score
    def top_k_words(self, k):
        top_k_rows = {
            k: v
            for k, v in sorted(
                self.row_mainword_scores.items(), key=lambda item: item[1], reverse=True
            )[:k]
        }
        top_k_columns = {
            k: v
            for k, v in sorted(
                self.column_mainword_scores.items(),
                key=lambda item: item[1],
                reverse=True,
            )[:k]
        }
        return top_k_rows, top_k_columns

<IPython.core.display.Javascript object>

In [6]:
# #adapting total score
# def total_score(self,main_word_and_location, crossing_words_and_locations):
#     letter_multipliers_array=np.copy(self.LETTER_MULTIPLIERS_ARRAY)
#     word_multipliers_array=np.copy(self.WORD_MULTIPLIERS_ARRAY)
#     # make the  multipliers==1 if there are pieces on them on the board;
#     # TODO: how permanent is the below?
#     bool_mask = (self.board != " ").astype(int)
#     letter_multipliers_array = np.where(bool_mask == 1, 1, letter_multipliers_array)
#     word_multipliers_array = np.where(bool_mask == 1, 1, word_multipliers_array)

#     main_word=main_word_and_location[0]
#     y=main_word_and_location[1][0]
#     x=main_word_and_location[1][1]
#     main_score = self.calculate_word_score(
#         main_word, y, x, letter_multipliers_array, word_multipliers_array
#     )
#     # TODO: ask if I should have transpose inside or outside the copy function here
#     # transpose the multiplier arrays to use to score the crossing words
#     T_word_multipliers = np.copy(word_multipliers_array.transpose())
#     T_letter_multipliers = np.copy(letter_multipliers_array.transpose())

#     # calculating the crossing words' total score
#     # need to switch the x and y here so the words don't go off the board
#     crossers_score = 0
#     for crosser in scored_crossing_words:
#         if len(crosser[0]) > 1:
#             crosser = list(crosser)
#             crosser[1], crosser[2] = crosser[2], crosser[1]
#             crosser_score = calculate_word_score(
#                 crosser, T_letter_multipliers, T_word_multipliers
#             )
#         else:
#             crosser_score = 0
#         crossers_score = crossers_score + crosser_score
#     total_score = main_score + crossers_score
#     return total_score

#     def valid_score_choices(
#         board, hand_letters, letter_multiplier_array, word_multiplier_array
#     ):
#         board=self.board
#         hand_letters=self.hand_letters
#         # make a transpose board
#         transpose_board = np.copy(board)
#         transpose_board = transpose_board.transpose()

#         # calculate the words/placement allowed by the rules of the game
#         valid_across_words_crosses, valid_column_words_crosses = valid_words_whole2(
#             board, transpose_board, hand_letters
#         )

#         # create a dictionary of each of these words along with their score
#         across_score_dict = {}
#         column_score_dict = {}
#         for word_crosses in valid_across_words_crosses:
#             word = word_crosses[0]
#             score = total_score(
#                 board, word_crosses, letter_multiplier_array, word_multiplier_array
#             )
#             across_score_dict[tuple(word)] = score

#         for word_crosses in valid_column_words_crosses:
#             column_word = word_crosses[0]
#             column_score = total_score(
#                 transpose_board,
#                 word_crosses,
#                 letter_multiplier_array,
#                 word_multiplier_array,
#             )
#             column_score_dict[tuple(column_word)] = column_score
#         return across_score_dict, column_score_dict



<IPython.core.display.Javascript object>

In [ ]:
# def valid_score_choices(
#     board, hand_letters, letter_multiplier_array, word_multiplier_array
# ):
#     # make a transpose board
#     transpose_board = np.copy(board)
#     transpose_board = transpose_board.transpose()

#     # calculate the words/placement allowed by the rules of the game
#     valid_across_words_crosses, valid_column_words_crosses = valid_words_whole2(
#         board, transpose_board, hand_letters
#     )

#     # create a dictionary of each of these words along with their score
#     across_score_dict = {}
#     column_score_dict = {}
#     for word_crosses in valid_across_words_crosses:
#         word = word_crosses[0]
#         score = total_score(
#             board, word_crosses, letter_multiplier_array, word_multiplier_array
#         )
#         across_score_dict[tuple(word)] = score

#     for word_crosses in valid_column_words_crosses:
#         column_word = word_crosses[0]
#         column_score = total_score(
#             transpose_board,
#             word_crosses,
#             letter_multiplier_array,
#             word_multiplier_array,
#         )
#         column_score_dict[tuple(column_word)] = column_score
#     return across_score_dict, column_score_dict

# def calculate_word_score(self, word, y, x, letter_multipliers_array, word_multipliers_array):
#     letter_multipliers = np.copy(
#         self.LETTER_MULTIPLIERS_ARRAY[y, x : x + len(word)]
#     )
#     word_multipliers = np.copy(self.WORD_MULTIPLIERS_ARRAY[y, x : x + len(word)])
#     scores = [self.TILE_SCORES[l] for l in word]
#     word_score = np.dot(letter_multipliers, scores)
#     word_score = word_score * np.prod(word_multipliers)
#     return word_score


In [6]:
# for edge row, I could find permutations of the surrounding area including spaces
# make a check surrroundings function for both axes

<IPython.core.display.Javascript object>

In [ ]:
sam = Scrabbler(board_array, sample_board_letters)
cw, _, _ = sam.complete_word_y_x(1, 1, ("a"))
samcross = sam.find_crosses("abcdfe", 12, 9)
# sam.edge_perms("s", 10, 7)
# {i for i in zip(*samcross)}
# valid_plays = {}
# for i in samcross:
#     valid_plays["a"] = 1  # ["b"][["c", "d"][0]] = ["e", "f"][1:3]
samcross

In [ ]:
sam = Scrabbler(board_array, sample_board_letters)
# samperms = sam.find_valid_perms()
# sam.find_edge_hooks(sam.valid_plays(sam.find_valid_perms()))
# todo edge hooks off by 1
# dict(sam.valid_plays(samperms))
# [i for i in sam.valid_plays()]
# [i for i in sam.hand_letter_permutations(1)][0]

# edge_hooks = sam.find_edge_hooks(sam.valid_plays(sam.find_valid_perms()))
# sam.board = sam.T_board
# edge_words = sam.valid_edge_words(edge_hooks)
# edge_plays = sam.valid_plays(edge_words)
# sam.unpack_dictionary(edge_plays)

valid_plays = sam.valid_plays(sam.find_valid_perms())
sam.unpack_dictionary(valid_plays)

In [7]:
# [i for i in sam.valid_plays()]
# [i for i in sam.valid_plays()["ea"]]
# sam.valid_plays()["ea"]
# sam.valid_plays()["ea"]

<IPython.core.display.Javascript object>

In [37]:
sam = Scrabbler(board_array, sample_board_letters)
# sam.valid_plays()
# sam.board_mask

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# the x's and y's are transposed here in the hook letters like they are automatically
# in the valid edge words method
vew = sam.valid_edge_words([["e", 4, 10], ["d", 9, 0], ["s", 9, 8]])
sam.unpack_dictionary(sam.valid_plays(vew))

In [ ]:
sam.all_board_words()

In [ ]:
#current structure:
{'mainword1': {
    (main1_y1,main1_x1): {
        'cross_word1': [(cross1_x1, cross1_y1), (cross1_x2, cross1_y2)], 
        'cross_word2': [(cross2_x1, cross2_y1),...]}, 
    (main1_y2,main1_x2): {...}}, 
 'main_word2':{similar to before}}
#hoped for:
{['mainword1', main1_y1,main1_x1]:(['cross_word1', cross1_x1, cross1_y1], 
                                   ['cross_word1', cross1_x2, cross1_y22], 
                                   ['cross_word2', cross2_x1, cross2_y1]),
 ['mainword2', main2_y1,main2_x1]: similar to above
}
 
 

In [ ]:
sam = Scrabbler(board_array, sample_board_letters)

valid_row_perms = sam.find_valid_perms()
valid_row_plays = sam.valid_plays(valid_row_perms)
# find the edge hooks
edge_hooks_for_columns = sam.find_edge_hooks(valid_row_plays)
# transpose
sam.board = sam.board.transpose()
# find the valid words (column words)
valid_column_perms = sam.find_valid_perms()
valid_column_plays = sam.valid_plays(valid_column_perms)
# find which edge words are valid from above
valid_column_edge_perms = sam.valid_edge_words(edge_hooks_for_columns)
valid_column_edge_plays = sam.valid_plays(valid_column_edge_perms)
# find the edge hooks for the rows
# edge_hooks_for_rows=
sam.unpack_dictionary(valid_row_plays)
valid_row_plays

In [22]:
# edge plays appears to be what's broken
# but it seems mostly right here. A few repeated crossing words. Could get fixed with a set
# Perhaps I'm not doing transpose at the right time in the main function
# sam = Scrabbler(board_array, sample_board_letters)
# samperms = sam.find_valid_perms()
# sam.find_edge_hooks(sam.valid_plays(sam.find_valid_perms()))
# todo edge hooks off by 1
# dict(sam.valid_plays(samperms))
# [i for i in sam.valid_plays()]
# [i for i in sam.hand_letter_permutations(1)][0]
# sam.board = sam.board.transpose()
# edge_hooks = sam.find_edge_hooks(sam.valid_plays(sam.find_valid_perms()))
# sam.board = sam.board.transpose()
# edge_words = sam.valid_edge_words(edge_hooks)
# edge_plays = sam.valid_plays(edge_words)
# sam.unpack_dictionary(edge_plays)

<IPython.core.display.Javascript object>

In [133]:
sam = Scrabbler(board_array, sample_board_letters)
# row_words,column_words=sam.all_board_words()
sam.valid_score_choices()

({('ta', (7, 11)): 2,
  ('tar', (7, 11)): 3,
  ('tao', (7, 11)): 3,
  ('tao', (8, 1)): 3,
  ('tas', (7, 11)): 3,
  ('tora', (7, 11)): 4,
  ('tola', (7, 11)): 5,
  ('tosa', (7, 11)): 4,
  ('tsar', (7, 11)): 4,
  ('stoa', (8, 1)): 10,
  ('of', (7, 2)): 5,
  ('to', (7, 11)): 2,
  ('to', (8, 2)): 2,
  ('fa', (7, 3)): 5,
  ('fa', (12, 13)): 8,
  ('orf', (7, 1)): 6,
  ('tor', (7, 11)): 3,
  ('oft', (7, 2)): 6,
  ('tot', (7, 11)): 3,
  ('oaf', (7, 1)): 6,
  ('arf', (7, 1)): 6,
  ('alf', (7, 1)): 7,
  ('aft', (7, 2)): 6,
  ('tat', (7, 11)): 3,
  ('rolf', (7, 0)): 8,
  ('trot', (7, 11)): 4,
  ('raft', (7, 1)): 7,
  ('trat', (7, 11)): 4,
  ('loft', (7, 1)): 8,
  ('loaf', (7, 0)): 8,
  ('torr', (7, 11)): 4,
  ('tort', (7, 11)): 4,
  ('orfs', (7, 1)): 7,
  ('tors', (7, 11)): 4,
  ('tolt', (7, 11)): 5,
  ('tots', (7, 11)): 4,
  ('tost', (7, 11)): 4,
  ('oafs', (7, 1)): 7,
  ('soft', (7, 1)): 7,
  ('sofa', (7, 1)): 7,
  ('saft', (7, 1)): 7,
  ('taro', (7, 11)): 4,
  ('taro', (8, 0)): 6,
  ('tart', (

<IPython.core.display.Javascript object>

In [135]:
sam.top_k_words(5)

({('nuns', (14, 8)): 27,
  ('rostral', (13, 0)): 26,
  ('slart', (13, 2)): 22,
  ('soral', (13, 2)): 22,
  ('solar', (13, 2)): 22},
 {('ewest', (8, 6)): 25,
  ('ewes', (8, 6)): 20,
  ('shorl', (4, 8)): 19,
  ('shoal', (4, 8)): 19,
  ('lash', (4, 6)): 18})

<IPython.core.display.Javascript object>

In [36]:
sam.total_score(("sen", (0, 10)), (("na", (12, 0)), ("so", (10, 0))))

9

<IPython.core.display.Javascript object>

In [128]:
print(np.where(board_array == "", " ", board_array))

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'd' 'e' 'a' 'd' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'z' 'o' 'n' 'a' ' ' 'w']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'a' ' ' ' ' 'i']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'r' 'i' 'm' 'a' ' ' 'l']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'o' 'b' 'e' 'y']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'j' 'u' 'r' 'a' 'l' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'b' 'e' 'e' 'p' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'f' ' ' ' ' 'y' 'e' 'w' 's' ' ' 't' ' ' ' ' ' ']
 [' ' ' ' ' ' 'o' ' ' ' ' ' ' 'l' 'e' 't' 'c' 'h' ' ' ' ' ' ']
 [' ' ' ' ' ' 'c' 'h' 'u' 'r' 'l' ' ' ' ' ' ' 'i' ' ' 'r' ' ']
 [' ' ' ' 'g' 'i' ' ' ' ' ' ' 'e' ' ' ' ' ' ' 'g' ' ' 'e' ' ']
 [' ' ' ' 'i' ' ' ' ' ' ' ' ' 'n' ' ' ' ' ' ' 'h' 'u' 'e' 'd']
 [' ' ' ' 'n' ' ' ' ' ' ' ' ' 'd' ' ' ' ' ' ' ' ' ' ' 'f' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'v' 'e' 'e' 'p' 's' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'n' 'u' 'n' ' ' ' ' ' ' ' ']]


<IPython.core.display.Javascript object>

In [52]:
print(
    np.where(
        np.transpose(board_array.copy()) == "", " ", np.transpose(board_array.copy())
    )
)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'g' 'i' 'n' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'f' 'o' 'c' 'i' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'u' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'y' ' ' 'r' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'b' 'e' 'l' 'l' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'e' 'w' 'e' ' ' ' ' ' ' ' ' ' ' 'n']
 [' ' 'z' ' ' 'r' ' ' 'j' 'e' 's' 't' ' ' ' ' ' ' ' ' 'v' 'u']
 ['d' 'o' ' ' 'i' ' ' 'u' 'p' ' ' 'c' ' ' ' ' ' ' ' ' 'e' 'n']
 ['e' 'n' 'a' 'm' 'o' 'r' ' ' 't' 'h' 'i' 'g' 'h' ' ' 'e' ' ']
 ['a' 'a' ' ' 'a' 'b' 'a' ' ' ' ' ' ' ' ' ' ' 'u' ' ' 'p' ' ']
 ['d' ' ' ' ' ' ' 'e' 'l' ' ' ' ' ' ' 'r' 'e' 'e' 'f' 's' ' ']
 [' ' 'w' 'i' 'l' 'y' ' ' ' ' ' ' ' ' ' ' ' ' 'd' ' ' ' ' ' ']]


<IPython.core.display.Javascript object>

In [9]:
sam.complete_word_y_x(11, 6, ["a", "b", "c"])

('dabc', 11, 6)

<IPython.core.display.Javascript object>

In [17]:
a = sam.row_permutations(7)
# output:
# {'dfdf':[[3,4]], 'dfkdf':[[5,7]],...}
# l = [[4, 3]]
# l.append(a["daosyews"])
# l
# [i for i in a.values()]
a["yewsatosd"]

[[7, 6]]

<IPython.core.display.Javascript object>

In [120]:
"et" in sam.WORDSET

True

<IPython.core.display.Javascript object>

In [19]:
a = {"a": 1, "b": 2, "c": 3}
a["a"] = {a["a"]: 2}
a
b = defaultdict(dict)
b["a"]["b"] = [1, 2]
b["a"]["c"] = [1, 2]

# b["r"] = {1: 2}
dict(b)

{'a': {'b': [1, 2], 'c': [1, 2]}}

<IPython.core.display.Javascript object>

In [59]:
d1 = {1: 2, 3: 4}
d2 = {1: 6, 3: 7}

dd = defaultdict(list)

for d in (d1, d2): # you can list as many input dicts as you want here
    for key, value in d.items():
        dd[key].append(value)

print(dict(dd).values())


dict_values([[2, 6], [4, 7]])


<IPython.core.display.Javascript object>

In [49]:
class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value


myDict = Vividict()

myDict[2000]["hello"] = [2, 3]
myDict

{2000: {'hello': [2, 3]}}

<IPython.core.display.Javascript object>

In [70]:
mydict2 = defaultdict(lambda: defaultdict(dict))
mydict2["a"][3, 2]["b"] = [2, 3]
mydict2["a"][3, 2]["c"] = [4, 3]

mydict2

defaultdict(<function __main__.<lambda>()>,
            {'a': defaultdict(dict, {(3, 2): {'b': [2, 3], 'c': [4, 3]}})})

<IPython.core.display.Javascript object>

In [104]:
dicitonary = {
    "agios": [[10, 1], [10, 1]],
    "taos": [[8, 1], [10, 5]],
    "ratos": [[8, 0], [10, 5]],
    "rotos": [[8, 0]],
    "roost": [[8, 1]],
    "roosa": [[8, 1]],
    "taros": [[8, 0], [10, 5]],
    "toros": [[8, 0]],
}

"agios" in dicitonary
("rotos", [[8, 0]]) in dicitonary.items()

[10, 1] in [[10, 1]]

True

<IPython.core.display.Javascript object>

In [ ]:
                                    # First, update the hook letters:
                                    if (
                                        len(complete_word) == perm_len + 1
                                    ):  # TODO: could also say len(hand_perm)==1
                                        # I'm transposing x and y here
                                        hook_with_position = [hand_perm[0], x, y]
                                        self.hook_letters_global.append(
                                            hook_with_position
                                        )
